In [26]:
import kmeans_combined_revised, simulatedData, time
from kmeans_combined_revised import kmeansplusplus, kmeansparallel, kmeans
from simulatedData import generate_data




In [35]:
#test on simulated data
k=50
R=1
d=15
l=100
data=generate_data(10,R)
data

array([[-2.14838773, -0.2633887 , -0.49821907, ...,  1.03409302,
         1.61503977,  0.97133414],
       [ 1.33594651, -0.51966662, -0.89920492, ...,  1.48862498,
        -0.75018261, -0.45887326],
       [ 0.50012471,  1.3292707 , -1.02077966, ...,  0.61863794,
        -1.02505497, -1.5782509 ],
       ..., 
       [ 0.50703875,  0.22581425,  2.36016058, ..., -0.09777885,
        -1.23833354, -1.41333935],
       [ 0.83696809, -1.64143642,  1.44469558, ...,  0.61236207,
         1.97956224,  0.83077656],
       [ 0.09693613,  1.18505513,  0.93147577, ..., -2.14432162,
         1.77613604,  4.61195676]])

In [28]:
%timeit kmeansplusplus(data,k,d)

1 loop, best of 3: 1min 19s per loop


In [37]:
%timeit kmeansparallel(data,6,l,d)

1 loop, best of 3: 10.1 s per loop


Optimization method 1:

Import kmeans_numba module

In [12]:
import kmeans_Numba_revised

In [13]:
%time
initial_3= kmeans_Numba_revised.kmeansparallel(data,10,10,15)
initial_3

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


array([[ -2.97283554e+00,   6.12885988e-01,  -2.05040807e+00,
         -5.94865655e-01,  -2.87283443e+00,   1.16661429e+00,
          8.07584677e-01,  -1.91555481e-01,   1.18299289e+00,
          1.20485396e+00,   1.31029273e+00,  -2.91787795e-01,
          1.61872655e+00,   5.99805421e-01,   5.67789582e-01],
       [ -2.23076786e-01,   1.30752385e-01,  -7.10183824e-01,
         -8.24215849e-02,   2.94371382e+00,   7.51694524e-01,
          1.11803847e+00,  -1.83682286e+00,  -9.00866530e-01,
         -1.04806766e+00,   1.73895864e+00,  -2.31896811e+00,
         -3.79518977e-01,   3.15507732e+00,   2.15207890e+00],
       [ -1.22610010e+00,   1.60597273e-01,  -2.43843318e+00,
         -9.94233562e-01,  -1.15978382e+00,   1.01685908e+00,
          3.79675546e+00,   1.54319134e+00,   1.05134976e+00,
          1.86108270e+00,   1.56123537e+00,  -1.08632521e+00,
          1.38465551e+00,   4.84920714e-01,   6.39446900e-01],
       [  8.59003104e-01,   1.03490951e+00,  -3.26725267e+00,
     

Optimization method 2:

Import kmeansCython module

In [9]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [15]:
%%cython
import numpy as np
cimport numpy as np
import pandas as pd
from pandas import DataFrame
import math

#distance square
def dist_sq_c(a, b):
    return np.sum((a-b)**2)
#minimum distance square for every point to the centroid
def point_sq_c(data,centroid):
    dist=[]
    for i in range(data.shape[0]):
        dist.append(min(dist_sq_c(data[i],c) for c in centroid))
    return dist
        

#calculate probability
def dist_prob_parallel_c(Dist,l):
    return l*Dist/np.sum(Dist)


#step 2: calculate the cost and number of iterations(log(cost))
def log_cost_c(data_copy,centroid):
    cost=np.sum(point_sq_c(data_copy,centroid))
    iteration=math.ceil(np.log(cost))
    return iteration

    
    
#calculate weights
#step 4: assign the weights
def weight_prob_c(data_copy, centroid):
    w_size=centroid.shape[0]
    w=np.zeros(w_size)
    for i in range(data_copy.shape[0]):
        index_w=np.argmin(list(dist_sq_c(data_copy[i],c) for c in centroid))
        w[index_w]=w[index_w]+1
    return w



#step 5: recluster the weighted points in C into k clusters
#reinitialize k centroids
def reassign_centroids_c(centroid,k,d,w):
    new_centroid=np.zeros([k,d])
    for cluster in range(k):
        #according to the weights from step 4, calculate the probability that a point is sampled from C
        prob_w=list(w/sum(w))
        #sample a new centroid
        new_index=np.random.choice(centroid.shape[0],1,prob_w)
        #store the new centroid
        new_centroid[cluster]=centroid[new_index]
        #delete the new centroid from the centroid
        centroid=np.delete(centroid,new_index,axis=0)
        #delete the correponding weight
        w=np.delete(w,new_index,axis=0)
    return new_centroid


def kmeansparallel_c(data, k, l, d):
    #step 1: sample a point uniformly at random from X
    index=int(np.random.choice(data.shape[0],1))
    centroid=np.array(data[index])
    data_copy=data.copy()
    data_copy=np.delete(data_copy,index,axis=0)
    
    #step 2: calculate number of iteration
    iteration= log_cost_c(data_copy,centroid)
    
    #step 3: Get initial Centroids C
    for number in range(iteration):
        #calculate phi_X(C)
        distance=point_sq_c(data_copy,centroid)
        #calculate the probability
        prob=dist_prob_parallel_c(distance,l).tolist()
        for n in range(data_copy.shape[0]):
            #if the probability is greater than the random uniform
            if prob[n]>np.random.uniform():
                #add the point to C
                centroid=np.vstack([centroid,np.array(data_copy[n])])
                #delete that point from the copy
                data_copy=np.delete(data_copy,n,axis=0)
    
    #step 4: calculate the weight probability
    w=weight_prob_c(data_copy,centroid)
    
    #step 5: recluster the weighted points in C into k clusters
    #reinitialize k centroids
    new_centroids=reassign_centroids_c(centroid,k,d,w)
    
    return new_centroids

    
#with the initialization of the centroids from the function kmeansplusplus
#plug in the original data(dataSet), initializtions(initial) and the dimension of the data(d)
def kmeans_c(dataSet, initial, k, d):
    centroids=initial
    # Initialize book keeping vars.
    iterations = 0
    oldCentroids = np.zeros(initial.shape)
    
    # Run the main k-means algorithm
    while not shouldStop_c(oldCentroids, centroids, iterations):
        # Save old centroids for convergence test. Book keeping.
        oldCentroids = centroids
        iterations += 1
        
        # Assign labels to each datapoint based on centroids
        l= getLabels_c(dataSet, centroids)
        
        # Assign centroids based on datapoint labels
        centroids = getCentroids_c(dataSet, l, k, d)
        
    # We can get the labels too by calling getLabels(dataSet, centroids)
    return centroids, np.array(l)
# Function: Should Stop
# -------------
# Returns True or False if k-means is done. K-means terminates either
# because it has run a maximum number of iterations OR the centroids
# stop changing.
def shouldStop_c(oldCentroids, centroids, iterations):
    if iterations > 50: return True
    return oldCentroids.all == centroids.all
# Function: Get Labels
# -------------
# Returns a label for each piece of data in the dataset. 
def getLabels_c(dataSet, centroids):
    # For each element in the dataset, chose the closest centroid. 
    # Make that centroid the element's label.
    l=[]
    for i in range(dataSet.shape[0]):
        #arg min as the label
        l.append(np.argmin(list(dist_sq_c(dataSet[i],c) for c in centroids)))
    return l
# Function: Get Centroids
# -------------
# Returns k random centroids, each of dimension n.
def getCentroids_c(dataSet, labels, k, d):
    # Each centroid is the arithmetic mean of the points that
    # have that centroid's label. Important: If a centroid is empty (no points have
    # that centroid's label) you should randomly re-initialize it.
    data_new = DataFrame(dataSet.copy())
    data_new['Labels'] = labels
    data_new = np.array(data_new.groupby(['Labels']).mean().iloc[:,:d])
    # if a centroid is empty, reinitialize it 
    if len(np.unique(labels))<k:
        diff=k-len(np.unique(labels))
        data_new=np.vstack([data_new,np.random.random([diff,d])])    
    return data_new

In [17]:
%time
initial_4= kmeansparallel_c(data,10,10,15)
initial_4

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


array([[ -7.05594099e-01,   1.52862527e+00,   1.49703714e+00,
         -6.09101002e-01,  -5.38510730e-01,   1.48150513e+00,
         -1.33389044e+00,   1.84757783e+00,  -8.99483656e-01,
          1.13351606e+00,  -7.31482268e-01,  -6.02266027e-02,
         -2.48307630e+00,  -3.56453513e-01,   4.03727512e-01],
       [ -9.93919905e-02,  -3.53303794e-01,   1.05408032e+00,
         -4.98714839e-01,  -2.82748490e+00,  -1.03186749e+00,
          2.10768746e+00,  -1.68080260e+00,   3.41034105e-01,
          1.37805843e+00,  -6.27243668e-02,   4.66947348e-01,
          5.27202657e-01,  -1.12528293e-01,  -8.77264353e-02],
       [ -1.10978872e+00,   1.85303916e+00,   1.75615171e+00,
          1.17032893e+00,   9.53215205e-01,  -9.83039623e-01,
          8.18520824e-01,   1.28759836e+00,  -6.88572450e-01,
         -3.84618203e-01,  -4.65957466e-01,  -4.29954447e-01,
          9.19388164e-01,   9.41094550e-01,   8.96015344e-01],
       [  8.61825914e-01,  -7.37077683e-01,   1.42312101e+00,
     